In [ ]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install opencv-python
!{sys.executable} -m pip install Pillow
!{sys.executable} -m pip install "tensorflow<2"

In [ ]:
from BoundingBox import *
from Camera import *
from CameraConfiguration import *
from Configuration import *
from CoordinateConverter import TopDownCoordinateConverter, BarycentricCoordinateConverter
from FieldState import *
from HumanDetector import *
from Point2 import *
from Size2 import *

In [ ]:
configuration = Configuration()
configuration.ModelPath = "Models/ssd_mobilenet_v1_coco_2018_01_28/frozen_inference_graph.pb"
configuration.ModelImageSize = Size2(1920, 1080)
# configuration.Cameras = [CameraConfiguration("rtsp://172.20.10.6:8080")]
# configuration.Cameras = [RTSPCameraConfiguration("rtsp://wowzaec2demo.streamlock.net/vod/mp4:BigBuckBunny_115k.mov", TopDownCoordinateConverter(Point2(0, 0), Size2(1870, 1465)))]
configuration.Cameras = [FileCameraConfiguration("BunningsTest10-3.mp4", BarycentricCoordinateConverter([Point2(0, 265), Point2(1392, 454), Point2(1193, 732), Point2(419, 1076)], [Point2(1050, 655), Point2(1483, 651), Point2(1477, 805), Point2(1192, 807)]))]
cameras = [cameraConfiguration.CreateCamera() for cameraConfiguration in configuration.Cameras]
# humanDetector = COCOTensorflowHumanDetector(modelPath=configuration.ModelPath, threshold=0.15)

yoloModelConfig = "Models/yolov3/yolov3.cfg"
yoloModelWeights = "Models/yolov3/yolov3.weights"
yoloModelClasses = "Models/yolov3/coco.names"
humanDetector = YoloV3OpenCVDetector(yoloModelConfig, yoloModelWeights, yoloModelClasses, 0.4)

In [ ]:
import matplotlib.pyplot

fieldStateHistory = []

while True:
    fieldState = FieldState()
    fieldState.Time = time.time()
    hasImages = False
    
    for camera in cameras:
        image = camera.GetFrame()
        fieldState.Images.append(image)
        if image is not None:
            hasImages = True
            image = cv2.resize(image, (configuration.ModelImageSize.Width, configuration.ModelImageSize.Height))
            # image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)

            boxes = humanDetector.Detect(image)

            for i in range(len(boxes)):
                box = boxes[i]
                globalPoint = camera.Configuration.ConvertToGlobalCoordinates(configuration.ModelImageSize, box.Center)
                if (globalPoint is not None):
                    fieldState.People.append(globalPoint)
                    cv2.rectangle(image, (box.TopLeft.X, box.TopLeft.Y), (box.BottomRight.X, box.BottomRight.Y), (255, 0, 0), 2)
                    # print(len(fieldStateHistory))
                    
            # matplotlib.pyplot.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            # matplotlib.pyplot.show()
            cv2.imshow("Live View", image)
            cv2.waitKey(1)
            floorPlanImage = fieldState.DrawImage()
            floorPlanImage = numpy.array(floorPlanImage)
            # floorPlanImage = floorPlanImage[:, :, ::-1].copy() 
            cv2.imshow("Floor Plan", floorPlanImage)
            cv2.waitKey(1)
        
    
    if hasImages:
        fieldStateHistory.append(fieldState)

In [ ]:
#modification for writing it out

import matplotlib.pyplot

fieldStateHistory = []

framesWritten = 0

while True:
    fieldState = FieldState()
    fieldState.Time = time.time()
    hasImages = False
    
    for camera in cameras:
        image = camera.GetFrame()
        
        fieldState.Images.append(image)
        if image is not None:
            hasImages = True
            image = cv2.resize(image, (configuration.ModelImageSize.Width, configuration.ModelImageSize.Height))

            boxes = humanDetector.Detect(image)

            for i in range(len(boxes)):
                box = boxes[i]
                try:
                    globalPoint = camera.Configuration.ConvertToGlobalCoordinates(configuration.ModelImageSize, box.Center)
                    if (globalPoint is not None):
                        fieldState.People.append(globalPoint)
                        cv2.rectangle(image, (box.TopLeft.X, box.TopLeft.Y), (box.BottomRight.X, box.BottomRight.Y), (255, 0, 0), 2)
                except:
                    pass
            
            floorPlanImage = fieldState.DrawImage()
            floorPlanImage = numpy.array(floorPlanImage)
            
            cv2.imwrite("Output/Video/" + str(framesWritten).zfill(7) + ".jpg", image)
            cv2.imwrite("Output/Floor Plan/" + str(framesWritten).zfill(7) + ".jpg", floorPlanImage)
            
            framesWritten += 1
    else:
        continue
    break
    
    if hasImages:
        fieldStateHistory.append(fieldState)

In [ ]:
# Use these FFPMEG commands to stitch the images into a video
# ffmpeg -framerate 24 -i C:\Users\ongan\Documents\RedVimana.Bunnings.ComputerVision\Output\Video\%07d.jpg Video.mp4
# ffmpeg -framerate 24 -i "C:\Users\ongan\Documents\RedVimana.Bunnings.ComputerVision\Output\Floor Plan\%07d.jpg" -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" "Floor Plan.mp4"